# Amazon Kendra 기반 RAG
> 이 노트북은  SageMaker Studio* **`Data Science 3.0`** kernel 및 ml.t3.medium 인스턴스에서 테스트 되었습니다.
---
### 중요
- 이 노트북은 Anthropic 의 Claude-v2 모델 접근 가능한 분만 실행 가능합니다. 
- 접근이 안되시는 분은 노트북의 코드와 결과 만을 확인 하시면 좋겠습니다.
- 만일 실행시에는 **"과금"** 이 발생이 되는 부분 유념 해주시기 바랍니다.

### 선수조건
- Kendra indexing이 되어 있어야 합니다


### 설정

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 (필요한 라이브러리가 설치되어 있는지 확인하고) Bedrock에 연결해야 합니다.

In [2]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

# 1. Bedrock Client 생성

In [2]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."
#os.environ["AWS_PROFILE"] = "bedrock_claude"

boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models())

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


# 2. Amazon Kendra 및 LLM 인 Claude-v2 모델 로딩

## LLM 로딩 (Claude-v2)

In [3]:
from langchain.llms.bedrock import Bedrock

In [4]:
# - create the Anthropic Model
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2"),
    client=boto3_bedrock,
    model_kwargs={'max_tokens_to_sample': 512}
)
llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7fc87b6a3310>, model_id='anthropic.claude-v2', model_kwargs={'max_tokens_to_sample': 512})

## Kendra indexing

## 선수 조건
- Kendra indexing이 되어 있어야 합니다.
- Kendra setting은 아래를 참고하세요.

"Create an Index" 선택
![nn](./img/kendra-1.png)

"Create Index" 선택
![nn](./img/kendra-2.png)

"Index name" 및 "Role" 선택
![nn](./img/kendra-3.png)

"Next" 선택
![nn](./img/kendra-4.png)

"next" 선택
![nn](./img/kendra-5.png)

생성된 index 선택
![nn](./img/kendra-6.png)

"데이터 소스이름" 및 "언어" 선택
![nn](./img/kendra-8.png)

"Role"선택
![nn](./img/kendra-9.png)

"S3 Bucket"선택 후 계속 진행
![nn](./img/kendra-10.png)

생성된 index에서 index-id 확인 <BR>
![nn](./img/kendra-14.png)

#### [중요] is_kendra == True 일시에 kendra_index_id 을 꼭 넣어 주세요.

In [12]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_kendra
##############################

kendra_index_id = "92778f99-5480-4a28-93fd-0075486848d1"

# 3. QnA

## 프로프트 템플릿 생성

In [13]:
from langchain import PromptTemplate

### [TIP] Prompt의 instruction의 경우 한글보다 **영어**로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [14]:
from utils.rag import create_bool_filter, run_RetrievalQA, run_RetrievalQA_kendra, show_context_used
from langchain.prompts import PromptTemplate

# prompt_template = """Human: 다음 문맥의 Information을 사용하여 고객 서비스 센터 직원처럼, 마지막 질문에 대한 목차 형식으로 답변을 제공하세요. 응답을 모르면 모른다고 말하고 응답을 만들려고 하지 마세요.

# {context}

# Question: {question}
# Assistant:"""

prompt_template = """
    
    Human: Use the following pieces of context to provide a concise answer to the question at the end. If you
    don't know the answer, just say that you don't know, don't try to make up an answer.
    
    {context}
    
    Question: {question}
    Assistant:"""



PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)


### LangChain RetrievalQA 를 통해 실행

* Atttribute filter
    - https://docs.aws.amazon.com/kendra/latest/APIReference/API_AttributeFilter.html#kendra-Type-AttributeFilter-ContainsAny
    - https://docs.aws.amazon.com/kendra/latest/APIReference/API_Retrieve.html

In [15]:
query = "클라우드에 보관 중인 인증서는 얼마나 유지 되나요?"

In [16]:
%%time
result = run_RetrievalQA_kendra(
    query=query,
    llm_text=llm_text,
    PROMPT=PROMPT,
    kendra_index_id=kendra_index_id,
    k=5,
    aws_region=aws_region,
    verbose=False
)

print("##################################")
print("query: ", query)
print("##################################")
print_ww(result['result'])
show_context_used(result['source_documents'])  

##################################
query:  클라우드에 보관 중인 인증서는 얼마나 유지 되나요?
##################################
 문서에 따르면, 정상적으로 이용하실 경우 금융인증서의 유효기간은 3년이며, 1년 동안 클라우드 이용 이력이 없는 경우는 안전한 이용을 위하여 클라우드 계정 및 저장된 모든
인증서가 삭제됩니다.
-----------------------------------------------
1. Chunk: 1165 Characters
-----------------------------------------------
Document Title: fsi_smart_faq_ko_preprocess.csv
Document Excerpt:
인증서를 저장한 매체에서는 인증서 비밀번호로 편리하게 이용할 수 있으나 다른 기기에서 이용하려면 기기마다 복사하거나 이동식디스크에 저장해서 휴대해야 하는 불편함이 있을 수
있습니다. 금융인증서는 금융인증서는 금융결제원의 클라우드에 저장하여 이용하는 인증서로 발급/이용 시에 클라우드에 접속이 필요합니다. 금융결제원 클라우드에 연결만 가능하다면 어디서든
편리하게 이용 가능하지만, PC나 USB, 휴대폰 등 다른 기기로 복사는 불가합니다. (유효기간 3년/발급 수수료 무료) ※ 클라우드 계정 연결을 위해 휴대폰을 통한 ARS,
SMS, 마이인포앱 인증 절차가 필요합니다. 인증서 신한은행 금융인증서 해외에서 발급할 수 있나요? 해외에서도 인증서 발급 업무는 가능합니다. 다만, 금융인증서 저장을 위한
금융결제원 클라우드 계정 생성 및 연결이 필요한 업무로 해외연락처를 통한 ARS인증이 진행됩니다. 금융인증서 신한은행 클라우드에 보관 중인 인증서는 얼마나 유지 되나요? 정상적으로
이용하실 경우 금융인증서의 유효기간은 3년이며, 1년 동안 클라우드 이용 이력이 없는 경우는 안전한 이용을 위하여 클라우드 계정 및 저장된 모든 인증서가 삭제됩니다. 금융인증서
신한은행